In [83]:
import pandas as pd
from datetime import datetime, timedelta
from Indicators import resample, detect_fractal_v2, SuperTrend, atr
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
script = "ethusd"

df = pd.read_csv(f"{script}_merged.csv", parse_dates=["date"])

df1 = df[df["date"] >= datetime(2025, 6, 1)].copy(deep=True)

dfx = resample(df1, "15T")

/Users/cryptosniper/Desktop/CryptoCode/vipin/Indicators.py:33: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.resample(timeframe, origin="start_day").agg(ohlcv_dict)
/Users/cryptosniper/Desktop/CryptoCode/vipin/Indicators.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  timeframe_delta = pd.to_timedelta(timeframe)


In [85]:
dfx.head()

,date,open,high,low,close,volume
0,2025-06-01 00:00:00,2528.45,2529.06,2518.41,2520.50,2974.521632
1,2025-06-01 00:15:00,2520.49,2523.32,2515.46,2517.70,3443.769333
2,2025-06-01 00:30:00,2517.80,2521.96,2515.90,2521.96,1348.412897
3,2025-06-01 00:45:00,2521.96,2526.51,2521.30,2526.08,2406.654460
4,2025-06-01 01:00:00,2526.08,2529.20,2518.34,2528.40,2097.741586


In [86]:
# detect_fractal_v2(dfx, 2)
# fractal_top	fractal_bottom	fractal_time_top	fractal_time_bottom

In [87]:
SuperTrend(dfx, 10, 3.0)

array([0, 1, 1, ..., 1, 1, 1], shape=(3093,))

In [ ]:
# apply indicators

dfx = detect_fractal_v2(dfx, 2)
dfx["supertrend"] = SuperTrend(dfx, 10, 3.0)
atr(dfx)
dfx.dropna(inplace=True, axis=0)


dfx["p_atr"] = dfx["ATR"].shift(1)
dfx["p_supertrend"] = dfx["supertrend"].shift(1)
dfx["n_open"] = dfx["open"].shift(-1)
dfx["n_date"] = dfx["date"].shift(-1)

dfx = dfx[dfx["date"] >= datetime(2025, 6, 30)].copy(deep=True)

dfx["date"] = pd.to_datetime(dfx["date"], format="%Y-%m-%d %H:%M:%S")
dfx["n_date"] = pd.to_datetime(dfx["n_date"], format="%Y-%m-%d %H:%M:%S")

In [89]:
dfx.head()

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open,n_date
2784,2025-06-30 00:00:00,2500.31,2508.69,2500.01,2505.97,7463.844874,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,8.68,8.39,0.29,8.68,11.772343,12.010215,-1.0,2505.97,2025-06-30 00:15:00
2785,2025-06-30 00:15:00,2505.97,2520.70,2500.68,2516.14,8332.719783,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,20.02,14.73,5.29,20.02,12.361461,11.772343,-1.0,2516.03,2025-06-30 00:30:00
2786,2025-06-30 00:30:00,2516.03,2524.59,2510.71,2518.80,8187.741679,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,13.88,8.45,5.43,13.88,12.469928,12.361461,-1.0,2518.70,2025-06-30 00:45:00
2787,2025-06-30 00:45:00,2518.70,2523.58,2514.49,2519.01,5623.404911,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,9.09,4.78,4.31,9.09,12.228505,12.469928,-1.0,2518.93,2025-06-30 01:00:00
2788,2025-06-30 01:00:00,2518.93,2520.40,2505.29,2509.14,8498.417271,2524.59,2411.06,2025-06-30 00:30:00,2025-06-29 21:00:00,-1,15.11,1.39,13.72,15.11,12.434326,12.228505,-1.0,2509.14,2025-06-30 01:15:00


In [90]:
buy_entries = (
    (dfx["close"] > dfx["fractal_top"])
    & (dfx["date"] > dfx["fractal_time_top"])
    & (dfx["p_supertrend"].astype("Int32") == -1)
)

sell_entries = (
    (dfx["close"] < dfx["fractal_bottom"])
    & (dfx["date"] > dfx["fractal_time_bottom"])
    & (dfx["p_supertrend"].astype("Int32") == 1)
)


dfx["Buy Entry Cond"] = buy_entries
dfx["Sell Entry Cond"] = sell_entries


dfx.head()

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open,n_date,Buy Entry Cond,Sell Entry Cond
2784,2025-06-30 00:00:00,2500.31,2508.69,2500.01,2505.97,7463.844874,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,8.68,8.39,0.29,8.68,11.772343,12.010215,-1.0,2505.97,2025-06-30 00:15:00,False,False
2785,2025-06-30 00:15:00,2505.97,2520.70,2500.68,2516.14,8332.719783,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,20.02,14.73,5.29,20.02,12.361461,11.772343,-1.0,2516.03,2025-06-30 00:30:00,False,False
2786,2025-06-30 00:30:00,2516.03,2524.59,2510.71,2518.80,8187.741679,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,13.88,8.45,5.43,13.88,12.469928,12.361461,-1.0,2518.70,2025-06-30 00:45:00,False,False
2787,2025-06-30 00:45:00,2518.70,2523.58,2514.49,2519.01,5623.404911,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,9.09,4.78,4.31,9.09,12.228505,12.469928,-1.0,2518.93,2025-06-30 01:00:00,False,False
2788,2025-06-30 01:00:00,2518.93,2520.40,2505.29,2509.14,8498.417271,2524.59,2411.06,2025-06-30 00:30:00,2025-06-29 21:00:00,-1,15.11,1.39,13.72,15.11,12.434326,12.228505,-1.0,2509.14,2025-06-30 01:15:00,False,False


In [91]:
# dfx.head(20)

In [93]:
main_df = dfx.drop_duplicates(
    subset=["fractal_top", "fractal_time_top", "Buy Entry Cond"], keep="first"
)
entry_df = main_df.drop_duplicates(
    subset=["fractal_bottom", "fractal_time_bottom", "Sell Entry Cond"], keep="first"
)

In [94]:
entry_df.tail(40)

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open,n_date,Buy Entry Cond,Sell Entry Cond
2784,2025-06-30 00:00:00,2500.31,2508.69,2500.01,2505.97,7463.844874,2525.08,2411.06,2025-06-29 23:00:00,2025-06-29 21:00:00,-1,8.68,8.39,0.29,8.68,11.772343,12.010215,-1.0,2505.97,2025-06-30 00:15:00,False,False
2802,2025-06-30 04:30:00,2501.69,2503.12,2496.34,2496.59,844.833726,2505.76,2495.66,2025-06-30 04:00:00,2025-06-30 03:15:00,-1,6.78,1.43,5.35,6.78,8.540568,8.675996,-1.0,2496.60,2025-06-30 04:45:00,False,False
2810,2025-06-30 06:30:00,2498.23,2500.01,2481.44,2483.72,7214.731576,2504.20,2492.31,2025-06-30 06:00:00,2025-06-30 05:00:00,-1,18.57,1.77,16.80,18.57,7.897083,7.076090,-1.0,2483.72,2025-06-30 06:45:00,False,False
2818,2025-06-30 08:30:00,2476.11,2476.63,2468.24,2468.34,2080.645479,2486.51,2455.58,2025-06-30 08:00:00,2025-06-30 06:45:00,1,8.39,0.52,7.87,8.39,8.882556,8.920445,1.0,2468.35,2025-06-30 08:45:00,False,False
2822,2025-06-30 09:30:00,2472.09,2473.87,2454.31,2459.05,5171.042690,2476.82,2466.17,2025-06-30 09:00:00,2025-06-30 08:45:00,1,19.56,1.77,17.79,19.56,9.333128,8.546445,1.0,2459.04,2025-06-30 09:45:00,False,True
2831,2025-06-30 11:45:00,2462.64,2465.04,2456.54,2458.30,951.438276,2468.39,2446.15,2025-06-30 11:15:00,2025-06-30 10:15:00,1,8.50,2.40,6.10,8.50,8.559691,8.564283,1.0,2458.27,2025-06-30 12:00:00,False,False
2836,2025-06-30 13:00:00,2459.86,2464.47,2459.75,2463.69,1069.438532,2465.00,2456.54,2025-06-30 12:30:00,2025-06-30 11:45:00,1,4.72,4.76,0.04,4.76,7.759924,7.990688,1.0,2463.68,2025-06-30 13:15:00,False,False
2840,2025-06-30 14:00:00,2462.00,2466.99,2449.21,2465.00,6853.295921,2474.45,2457.01,2025-06-30 13:30:00,2025-06-30 12:45:00,1,17.78,5.13,12.65,17.78,9.225686,8.567662,1.0,2465.20,2025-06-30 14:15:00,False,False
2843,2025-06-30 14:45:00,2459.05,2460.79,2433.10,2437.80,16450.894032,2469.25,2449.21,2025-06-30 14:15:00,2025-06-30 14:00:00,1,27.69,1.71,25.98,27.69,11.100365,9.824239,1.0,2437.80,2025-06-30 15:00:00,False,True
2849,2025-06-30 16:15:00,2476.71,2478.06,2460.83,2463.24,5803.543360,2487.50,2433.10,2025-06-30 15:45:00,2025-06-30 14:45:00,1,17.23,1.13,16.10,17.23,13.293236,12.990408,1.0,2463.11,2025-06-30 16:30:00,False,False


In [95]:
entry_df.dropna(inplace=True, axis=0)
entry_df.reset_index(inplace=True)

/var/folders/t1/hprdk945653_456srgszhpw80000gr/T/ipykernel_2709/1342345424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry_df.dropna(inplace=True, axis=0)


In [96]:
p = entry_df.iloc[0]["date"] - timedelta(days=1)
p

Timestamp('2025-06-29 00:00:00')

In [98]:
dfx.at[0, "date"]

Timestamp('2025-06-30 00:00:00')

In [ ]:
## flags
prev_fractal = 0
prev_exit_time = entry_df.at[0, "date"]
EntryPrice = 0
entry = False
side = ""
final_sheet = []
run_time = entry_df.at[0, "date"]


for i in range(len(entry_df) - 2):

    EntryTime = entry_df.at[i, "n_date"]

    if entry_df.at[i, "n_date"] < prev_exit_time or :
        continue

    FractalTop = entry_df.at[i, "fractal_top"]
    FractalBottom = entry_df.at[i, "fractal_bottom"]
    FractalTopTime = entry_df.at[i, "fractal_time_top"]
    FractalBottomTime = entry_df.at[i, "fractal_time_bottom"]
    
    atr = entry_df.at[i, "p_atr"]
    spt = entry_df.at[i, "p_supertrend"]

    if entry_df.at[i, "Buy Entry Cond"] and FractalTopTime >= prev_exit_time:
        entry = True
        EntryPrice = entry_df.at[i,"n_open"]
        side = "buy"
        prev_fractal = FractalTop
        prev_exit_time = entry_df["date"][i]

    if entry_df.at[i, "Sell Entry Cond"] and FractalBottomTime >= prev_exit_time:
        entry = True
        EntryPrice = entry_df.at[i,"n_open"]
        side = "sell"
        prev_fractal = FractalBottom
        prev_exit_time = entry_df["date"][i]

    if entry and side == "buy":
        # print("Buy trade")
        sl = EntryPrice - (1 * atr)
        tg = EntryPrice + (2 * atr)
        sl_flt = dfx[(dfx["date"] > EntryTime) & (dfx["low"] < sl)]
        tg_flt = dfx[(dfx["date"] > EntryTime) & (dfx["high"] > tg)]

        if not sl_flt.empty and not tg_flt.empty:
            # check who is first
            first_sl = sl_flt.iloc[0]["date"]
            first_tg = tg_flt.iloc[0]["date"]

            if first_sl < first_tg:
                ExitPrice = sl
                ExitTime = first_sl
                ExitType = "SL"
            else:
                ExitPrice = tg
                ExitTime = first_tg
                ExitType = "TG"

        if sl_flt.empty and not tg_flt.empty:
            ExitPrice = tg
            ExitTime = tg_flt.iloc[0]["date"]
            ExitType = "TG"

        if tg_flt.empty and not sl_flt.empty:
            ExitPrice = sl
            ExitTime = sl_flt.iloc[0]["date"]
            ExitType = "SL"

        prev_exit_time = ExitTime
        doc = {
            "EntryTime": EntryTime,
            "EntryPrice": EntryPrice,
            "Side": side,
            "FractalPrice": FractalTop,
            "FractalTime": FractalTopTime,
            "SuperTrend": spt,
            "Atr": atr,
            "StopLoss": sl,
            "Target": tg,
            "ExitType": ExitType,
            "ExitPrice": ExitPrice,
            "ExitTime": ExitTime,
        }

        final_sheet.append(doc)

    if entry and side == "sell":
        # print("Sell trade")
        sl = EntryPrice + (1 * atr)  # SL above entry for sell
        tg = EntryPrice - (2 * atr)  # TG below entry for sell
        sl_flt = dfx[(dfx["date"] > EntryTime) & (dfx["high"] > sl)]
        tg_flt = dfx[(dfx["date"] > EntryTime) & (dfx["low"] < tg)]

        if not sl_flt.empty and not tg_flt.empty:
            # check who is first
            first_sl = sl_flt.iloc[0]["date"]
            first_tg = tg_flt.iloc[0]["date"]

            if first_sl < first_tg:
                ExitPrice = sl
                ExitTime = first_sl
                ExitType = "SL"
            else:
                ExitPrice = tg
                ExitTime = first_tg
                ExitType = "TG"

        if sl_flt.empty and not tg_flt.empty:
            ExitPrice = tg
            ExitTime = tg_flt.iloc[0]["date"]
            ExitType = "TG"

        if tg_flt.empty and not sl_flt.empty:
            ExitPrice = sl
            ExitTime = sl_flt.iloc[0]["date"]
            ExitType = "SL"

        prev_exit_time = ExitTime
        doc = {
            "EntryTime": EntryTime,
            "EntryPrice": EntryPrice,
            "Side": side,
            "FractalPrice": FractalTop,
            "FractalTime": FractalTopTime,
            "SuperTrend": spt,
            "Atr": atr,
            "StopLoss": sl,
            "Target": tg,
            "ExitType": ExitType,
            "ExitPrice": ExitPrice,
            "ExitTime": ExitTime,
        }

        final_sheet.append(doc)

In [100]:
FractalTop, FractalBottom, FractalTopTime, FractalBottomTime, atr, entry, side

(np.float64(2600.73),
 np.float64(2583.74),
 Timestamp('2025-07-02 21:45:00'),
 Timestamp('2025-07-02 21:15:00'),
 np.float64(12.32143323852862),
 True,
 'buy')

In [ ]:
sheet = pd.DataFrame(final_sheet)
sheet.head()

,EntryTime,EntryPrice,Side,FractalPrice,FractalTime,SuperTrend,Atr,StopLoss,Target,ExitType,ExitPrice,ExitTime
0,2025-06-30 09:45:00,2459.04,sell,2476.82,2025-06-30 09:00:00,1.0,8.546445,2467.586445,2441.947109,SL,2467.586445,2025-06-30 11:15:00
1,2025-06-30 12:00:00,2459.04,sell,2468.39,2025-06-30 11:15:00,1.0,8.564283,2467.604283,2441.911434,SL,2467.604283,2025-06-30 13:15:00
2,2025-06-30 13:15:00,2459.04,sell,2465.00,2025-06-30 12:30:00,1.0,7.990688,2467.030688,2443.058625,SL,2467.030688,2025-06-30 13:30:00
3,2025-06-30 14:15:00,2459.04,sell,2474.45,2025-06-30 13:30:00,1.0,8.567662,2467.607662,2441.904677,TG,2441.904677,2025-06-30 14:45:00
4,2025-06-30 15:00:00,2459.04,sell,2469.25,2025-06-30 14:15:00,1.0,9.824239,2468.864239,2439.391521,TG,2439.391521,2025-06-30 15:15:00


In [ ]:
import numpy as np

sheet["pnl"] = np.where(
    sheet["Side"] == "buy",
    sheet["ExitPrice"] - sheet["EntryPrice"],
    sheet["EntryPrice"] - sheet["ExitPrice"],
)

In [71]:
sheet.tail(12)

,EntryTime,EntryPrice,Side,FractalPrice,FractalTime,SuperTrend,Atr,StopLoss,Target,ExitType,ExitPrice,ExitTime,pnl
11,2025-07-01 10:30:00,2483.27,buy,2461.38,2025-07-01 10:00:00,1.0,7.061366,2476.208634,2497.392731,SL,2476.208634,2025-07-01 10:45:00,-7.061366
12,2025-07-01 14:45:00,2422.73,sell,2456.24,2025-07-01 14:15:00,1.0,11.689062,2434.419062,2399.351876,TG,2399.351876,2025-07-01 18:45:00,23.378124
13,2025-07-01 23:00:00,2397.63,sell,2412.42,2025-07-01 22:30:00,1.0,9.728595,2407.358595,2378.172809,SL,2407.358595,2025-07-02 00:15:00,-9.728595
14,2025-07-02 01:00:00,2397.63,sell,2409.11,2025-07-02 00:30:00,1.0,8.600136,2406.230136,2380.429727,TG,2380.429727,2025-07-02 01:15:00,17.200273
15,2025-07-02 02:30:00,2397.63,sell,2417.28,2025-07-02 02:00:00,1.0,9.480106,2407.110106,2378.669787,SL,2407.110106,2025-07-02 02:45:00,-9.480106
16,2025-07-02 03:15:00,2397.63,sell,2417.28,2025-07-02 02:00:00,-1.0,8.921893,2406.551893,2379.786215,SL,2406.551893,2025-07-02 03:30:00,-8.921893
17,2025-07-02 04:30:00,2397.63,sell,2431.61,2025-07-02 03:15:00,-1.0,8.180766,2405.810766,2381.268468,SL,2405.810766,2025-07-02 04:45:00,-8.180766
18,2025-07-02 06:45:00,2450.81,buy,2448.42,2025-07-02 06:00:00,-1.0,7.807179,2443.002821,2466.424358,SL,2443.002821,2025-07-02 07:00:00,-7.807179
19,2025-07-02 09:45:00,2450.81,buy,2452.70,2025-07-02 09:15:00,-1.0,6.737736,2444.072264,2464.285472,SL,2444.072264,2025-07-02 10:00:00,-6.737736
20,2025-07-02 10:45:00,2450.81,buy,2452.70,2025-07-02 09:15:00,-1.0,6.144591,2444.665409,2463.099181,SL,2444.665409,2025-07-02 11:45:00,-6.144591


In [34]:
sheet.to_excel("SPT_FRA_BACKTEST.xlsx", index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [36]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
dfx.tail()

NameError: name 'dfx' is not defined

In [ ]:
from datetime import datetime
import time

tm = [0, 15, 30, 45]

while True:
    dt_m = datetime.now().minute
    if dt_m in tm:
        print("Time", dt_m)
        time.sleep(60)
        break

Time 30
